In [50]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
import pymongo

In [51]:
def finding_raw_urls_from_base(url):
    service = Service("./chromedriver/chromedriver.exe")
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    
    WebDriverWait(driver, 60).until(
        EC.presence_of_all_elements_located((By.TAG_NAME, 'a'))
    )
    
    html = driver.page_source
    driver.quit()
    
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a', href=True)
    raw_urls = []
    for link in links:
        if link['href'].startswith("https://www.moneycontrol.com/news/business/"):
            raw_urls.append(link['href'])
    return list(set(raw_urls))
    

In [52]:
raw_urls = finding_raw_urls_from_base("https://www.moneycontrol.com/news/business/stocks/")
# raw_urls = finding_raw_urls_from_base("https://www.moneycontrol.com/news/business/stocks/page-2")

In [53]:
raw_urls

['https://www.moneycontrol.com/news/business/banks/',
 'https://www.moneycontrol.com/news/business/stocks/subscribe-for-stallion-india-fluorochemicals-ltd-ipo-anand-rathi-12911972.html',
 'https://www.moneycontrol.com/news/business/stocks/page-2/',
 'https://www.moneycontrol.com/news/business/markets/rvnl-shares-jump-9-after-winning-rs-3-622-crore-order-from-bsnl-12911961.html',
 'https://www.moneycontrol.com/news/business/markets/trade-spotlight-how-should-you-trade-biocon-equitas-small-finance-bank-minda-corporation-kfin-technologies-blue-star-and-others-on-january-16-12911774.html',
 'https://www.moneycontrol.com/news/business/stocks/page-4/',
 'https://www.moneycontrol.com/news/business/stocks/page-7/',
 'https://www.moneycontrol.com/news/business/oil-little-changed-as-falling-us-stockpiles-outweigh-soft-demand-outlook-12910813.html',
 'https://www.moneycontrol.com/news/business/personal-finance/income-tax-filing/',
 'https://www.moneycontrol.com/news/business/companies/ixigo-are-y

In [54]:
stock_urls =[]
market_urls = []
next_page_urls = []
other_urls = []
ipo_urls = []
def extract_urls(all_urls):
    for link in raw_urls:
        if link.startswith("https://www.moneycontrol.com/news/business/stocks/page"):
            next_page_urls.append(link)
        elif link.startswith("https://www.moneycontrol.com/news/business/stocks/"):
            stock_urls.append(link)
        elif link.startswith("https://www.moneycontrol.com/news/business/markets/"):
            market_urls.append(link)
        elif link.startswith("https://www.moneycontrol.com/news/business/ipo/"):
            ipo_urls.append(link)
        else:
            other_urls.append(link)

In [55]:
extract_urls(raw_urls)

In [56]:
stock_urls

['https://www.moneycontrol.com/news/business/stocks/subscribe-for-stallion-india-fluorochemicals-ltd-ipo-anand-rathi-12911972.html',
 'https://www.moneycontrol.com/news/business/stocks/',
 'https://www.moneycontrol.com/news/business/stocks/top-gainers-and-losers-today-stocks-that-moved-the-most-on-january-15-12911359.html',
 'https://www.moneycontrol.com/news/business/stocks/agri-picks-report-january-16-2025-geojit-financial-services-12911862.html',
 'https://www.moneycontrol.com/news/business/stocks/asian-stocks-gain-after-us-core-inflation-cools-12911792.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-angel-one-target-of-rs-3200-motilal-oswal-12911362.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-hdfc-amc-target-of-rs-5200-motilal-oswal-12911351.html',
 'https://www.moneycontrol.com/news/business/stocks/trent-shares-rise-nearly-5-after-elara-securities-gives-buy-rating-12911167.html']

In [57]:

regex = r'https:\/\/www\.moneycontrol\.com\/news\/business\/stocks\/[^\/\s]+(?:\.[a-z]{2,6})(?:[\/\?].*)?'

final_stocks_urls = [url for url in stock_urls if re.match(regex, url)]

In [58]:
final_stocks_urls

['https://www.moneycontrol.com/news/business/stocks/subscribe-for-stallion-india-fluorochemicals-ltd-ipo-anand-rathi-12911972.html',
 'https://www.moneycontrol.com/news/business/stocks/top-gainers-and-losers-today-stocks-that-moved-the-most-on-january-15-12911359.html',
 'https://www.moneycontrol.com/news/business/stocks/agri-picks-report-january-16-2025-geojit-financial-services-12911862.html',
 'https://www.moneycontrol.com/news/business/stocks/asian-stocks-gain-after-us-core-inflation-cools-12911792.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-angel-one-target-of-rs-3200-motilal-oswal-12911362.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-hdfc-amc-target-of-rs-5200-motilal-oswal-12911351.html',
 'https://www.moneycontrol.com/news/business/stocks/trent-shares-rise-nearly-5-after-elara-securities-gives-buy-rating-12911167.html']

In [59]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["new_db"]
mycol = mydb['raw_news']

In [60]:
service = Service("./chromedriver/chromedriver.exe")
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36")

driver = webdriver.Chrome(service=service, options= options)

In [61]:
def extract_data(url):  
    driver.get(url)
    
    WebDriverWait(driver, 60).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')    
    news = {}
    
    title = soup.find('h1', class_ = "article_title")
    news.update({"title":f"{title.text.strip()}"})
    
    desc = soup.find('h2', class_ = 'article_desc')
    news.update({"desc":f"{desc.text.strip()}"})
    
    date_time_div = soup.find('div', class_="article_schedule")
    if date_time_div:
        span_tag = date_time_div.find('span')
        date = span_tag.text.strip()
        news.update({"date":f"{date}"})
    
    datetime = date_time_div.text.strip()
    news.update({"datetime":f"{datetime}"})
    
    paragraphs_list = []
    paragrphs_div = soup.find('div', class_ ="content_wrapper")
    if paragrphs_div:
        paragraph_tags = paragrphs_div.find_all('p')
        for p in paragraph_tags:
            para_text = p.text.strip()            
            if len(para_text) < 50 :
                continue            
            if re.search(r"(click\s+here|disclaimer|modal|window|advertisement|investment\s+tips)", para_text, re.IGNORECASE):
                continue            
            paragraphs_list.append(para_text)         
            
    news.update({"content": paragraphs_list})
    
    stock_name = soup.find('a', class_="stock-name")
    if stock_name:
        news.update({"stock_name":f"{stock_name.text.strip()}"})
            
    # driver.quit()   
    return news

In [62]:
for url in final_stocks_urls :
    data = extract_data(url)
    x = mycol.insert_one(data)
    print(x.inserted_id)

67889a42761c49c0fb36fc99
67889a9e761c49c0fb36fc9a
67889ae3761c49c0fb36fc9b
67889b28761c49c0fb36fc9c
67889b6f761c49c0fb36fc9d
67889ba4761c49c0fb36fc9e
67889be2761c49c0fb36fc9f


In [63]:
driver.quit()   